In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error


c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
df = pd.read_csv('output.csv', index_col=0).reset_index(drop=True).set_index('itemid')
df['more_than_10c'] = df['price'] > 10.0
df = df[df['price'] > 0.1]

In [ ]:
X = df.drop(['price', 'basetype', 'corrupted', 'timestamp', 'more_than_10c'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
#dtrain = xgb.DMatrix(data=X_train, label=y_train)
#dtest = xgb.DMatrix(data=X_test, label=y_test)
#param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
#num_round = 10
#evallist = [(dtest, 'eval'), (dtrain, 'train')]

reg = xgb.XGBRegressor(tree_method = 'gpu_hist')
reg.fit(X_train, y_train)

ypred = reg.predict(X_test)

mae = median_absolute_error(y_test, ypred)
print(mae)

In [3]:
def create_lexicon(df, subset=None):
    if subset == None:
        raise ValueError("Define a subset to create lexicon on.")
    mod_dict = {"mod": [""]}
    mods_df = pd.DataFrame.from_dict(mod_dict)
    for _, item in df.iterrows():
        if item[subset] is not None:
            for mod in item[subset]:
                mod_no_digits = strip_digits(mod)
                if mod_no_digits not in mods_df.values:
                    mods_df.loc[len(mods_df.index)] = mod_no_digits
    mods_df["mod"].replace("", np.nan, inplace=True)
    mods_df.dropna(inplace=True)
    return [s + " (" + subset + ")" for s in mods_df["mod"].tolist()]
    
# create affix lexica
print("creating implicit lexicon...")
implicit_lexicon = create_lexicon(input_df, "implicit")
print("creating explicit lexicon...")
explicit_lexicon = create_lexicon(input_df, "explicit")
print("creating fractured lexicon...")
fracturedmods_lexicon = create_lexicon(input_df, "fracturedmods")

# merge info and affix lexica
print("merging lexica...")
info_lexicon = ["itemid", "price", "basetype", "ilvl", "corrupted", "timestamp"]
output_df = pd.DataFrame(
    columns=info_lexicon
    + explicit_lexicon
    + implicit_lexicon
    + fracturedmods_lexicon
)
lexica = [info_lexicon, implicit_lexicon, explicit_lexicon, fracturedmods_lexicon]
item_dict = {}
for lexicon in lexica:
    for mod in lexicon:
        item_dict[mod] = []
print(item_dict)

In [12]:
import pandas as pd
test = pd.read_pickle('data/output.pkl')

In [15]:
[x for x in test.columns if 'fractured' in x]

[' to Cold Resistance (fracturedmods)',
 ' to Fire Resistance (fracturedmods)',
 'Curse Enemies with Conductivity on Hit with  increased Effect (fracturedmods)',
 'Adds  to  Physical Damage to Attacks (fracturedmods)',
 ' to Level of Socketed Gems (fracturedmods)',
 ' to Lightning Resistance (fracturedmods)',
 ' to Strength (fracturedmods)',
 ' to maximum Life (fracturedmods)',
 ' increased Cold Damage (fracturedmods)',
 ' to Dexterity (fracturedmods)',
 ' increased Life Reservation Efficiency of Skills (fracturedmods)',
 ' to Intelligence (fracturedmods)',
 ' to maximum Energy Shield (fracturedmods)',
 ' increased Attack Speed (fracturedmods)',
 'Curse Enemies with Flammability on Hit with  increased Effect (fracturedmods)',
 ' increased Fire Damage (fracturedmods)',
 ' to all Elemental Resistances (fracturedmods)',
 'Adds  to  Lightning Damage to Attacks (fracturedmods)',
 ' to Chaos Resistance (fracturedmods)',
 ' increased Lightning Damage (fracturedmods)',
 ' increased Elemental D

In [9]:
df = (
    pd.read_pickle("data/output.pkl")
    .drop(columns=["timestamp"])
)
df["corrupted"] = df["corrupted"].astype(float)
test = pd.concat(
        [df.drop(columns=["price", "basetype"]), pd.get_dummies(df["basetype"])],
        axis=1,
    )
test1 = pd.DataFrame(columns=test.columns)

In [10]:
test1.to_pickle('data/lexicon.pkl')